In [ ]:
pip install openai jupyter python-dotenv -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

load_dotenv(find_dotenv(), override=True)
client = OpenAI()

In [3]:
def create_meals(
    ingredients,
    kcal=2000,
    exact_ingredients=False,
    output_format="text",
    model="gpt-3.5-turbo",
    system_role="Você é um cozinheiro habilidoso com a expertise de um chef.",
    temperature=1,
    extra=None,
):
    prompt = f"""
    Crie um plano diário de alimentação para café da manhã, almoço e janta baseado nos seguintes ingedientes: 
    {ingredients}
    Seu output tem ser no formato {output_format}
     Siga as instruções abaixo cuidadosamente
     ### Instruções:
    1. {"Só use os ingredientes providos com sal, pimenta e especiarias." if exact_ingredients else "Fique livre para incorporar os ingredientes dados como base e adicione outros ingredientes se você considerar necessario para melhorar o sabor, valor nutricional ou apelo geral das receitas"}
    2. Especifique a quantidade exata de cada ingrediente
    3. Tenha certeza que a quantidade de calorias está abaixo de {kcal}
    4. Para cada refeição, explique cada receita, passo a passo, com frases simples e claras. Use marcadores ou numeroes para organizar os passos
    5. Para cada refeição, especifique o total de calorias e o quantas pessoas servem
    6. Para cada refeição, dê um titulo conciso e descritivo que resuma os ingredientes principais e os sabores. O titulo deve ser um prompt valido pro DALL-E para gerar uma imagem original para a receita
    7. Para cada receita, diga o tempo total de preparo e cozinhamento.
    {"8. Se possivel as refeições devem ser:" + extra if extra else ""}

    Antes de responder, tenha certeza que seguiu as instruções listadas acima (pontos 1 a 7 ou 8)
    A ultima linha da sua resposta deve ser uma string que contém APENAS o titulo das refeições com uma virgula entre cada e nada mais
    \n Brocolis e ovo mexido, frango grelhado e vegetais, peixe cozido com salada de repolho
"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content

In [4]:
foods = "azeite extra virgem, grãos, frutas frescas e vegetais, nozes, peixe, ovo, comidas fermetadas, mel"
output = create_meals(ingredients=foods)
print(output)

### Café da Manhã:

#### Salmão Defumado com Salada de Abacate e Nozes
- **Ingredientes:**
  - 100g de salmão defumado
  - 1 abacate
  - 20g de nozes
  - Azeite extra virgem
  - Sal e pimenta a gosto

- **Instruções:**
  1. Corte o salmão defumado em pedaços pequenos.
  2. Corte o abacate em cubos e misture com o salmão.
  3. Adicione as nozes e tempere com azeite, sal e pimenta.
  4. Misture tudo suavemente e sirva.

- **Total de Calorias:** 350 calorias
- **Serve:** 1 pessoa
- **Tempo Total:** 10 minutos

### Almoço:

#### Salada Fresca de Quinoa com Vegetais e Ovo Cozido
- **Ingredientes:**
  - 50g de quinoa
  - Vegetais variados (como tomate, pepino, cenoura)
  - 1 ovo
  - Azeite extra virgem
  - Vinagre balsâmico
  - Sal e pimenta a gosto

- **Instruções:**
  1. Cozinhe a quinoa conforme as instruções da embalagem e reserve.
  2. Corte os vegetais em pedaços pequenos e misture com a quinoa.
  3. Cozinhe o ovo e corte em fatias.
  4. Tempere a salada com azeite, vinagre balsâmico, 